```
Day 9 / 豆瓣 TOP250 数据分析        
    保存
        使用 openpyxl 写入 excel 表格
        插入图片
        插入超链接
    简单数据统计
        年平均分
        国家平均分
        各分段电影数
        各类别电影均分
        各类别在总体的占比
        按国别统计电影数在总体的占比
    可视化：pyecharts
        安装
        Bar / Line / Pie 简单示例
        接入生成的数据
        修改配色
        配置标题、图例和坐标轴
```

In [2]:
import openpyxl
from openpyxl.drawing.image import Image as ExcelImage
from openpyxl.drawing.spreadsheet_drawing import AnchorMarker, TwoCellAnchor

def insert_image_to_sheet(image_path, row, col):
    # 断言：确保列号在 0-25 之间
    assert 0 <= col <= 25
    # 将列号转换为字母
    col_index = chr(ord('A') + col)

    # 输出图片路径
    print(image_path)

    # 创建 ExcelImage 对象
    img = ExcelImage(image_path)
    # 获取图片高度和宽度
    height = img.height
    width = img.width
    
    # 调整列宽，列宽单位为 character
    ws.column_dimensions[col_index].width = width / 8
 
    # 调整行高，行高单位为 point
    ws.row_dimensions[row + 1].height = height / 4 * 3
    
    # 设置锚点，使图片随单元格一起移动
    # 创建起始和结束锚点（左上 和 右下）
    from_anchor = AnchorMarker(col, 0, row, 0)
    to_anchor = AnchorMarker(col + 1, 0, row + 1, 0)
    # 根据起始锚点和结束锚点，生成 左上+右下 组合定位锚点
    img.anchor = TwoCellAnchor('twoCell', from_anchor, to_anchor)
    
    # 将图片添加到工作表
    ws.add_image(img)

def make_cell_as_hyperlink(row, col, link, text=None):
    # 设置单元格超链接的链接地址
    ws.cell(row, col).hyperlink = link
    # 如果提供了文本，则设置单元格值为文本
    if text is not None:
        ws.cell(row, col).value = text
    # 设置单元格样式为超链接样式
    ws.cell(row, col).style = "Hyperlink"

In [ ]:
# 爬取、提取
import requests, bs4, time, json, re

# 定义了一个请求头部，包含 UserAgent 用于模拟浏览器访问
headers = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.188' }

# 定义了一个函数，用于获取指定页面的HTML内容
# 参数：page_num 表示要获取的页面的页码，从0开始计数
def get_html_of_page(page_num):
    # 等待1秒，避免请求过于频繁
    time.sleep(1)
    
    # 发送一个GET请求到指定URL，使用之前定义的请求头部，以及包含查询参数的字典
    resp = requests.get("https://movie.douban.com/top250", headers=headers, params={
        "start": page_num * 25,  # 每页显示25个电影，计算起始电影的序号
        "filter": ""  # 过滤器为空，获取所有电影
    })
    
    # 如果响应状态码不为200，抛出一个异常
    if resp.status_code != 200:
        raise ValueError(f"Failed to get page: {page_num}")
    
    # 返回响应的HTML内容
    return resp.text

# 定义了一个函数，用于获取指定URL的HTML内容
# 参数：url 表示要获取内容的URL地址
def get_html_of_url(url):
    # 等待1秒，避免请求过于频繁
    time.sleep(1)
    
    # 发送一个GET请求到指定URL，使用之前定义的请求头部
    resp = requests.get(url, headers=headers)
    
    # 如果响应状态码不为200，抛出一个异常
    if resp.status_code != 200:
        raise ValueError(f"Failed to get url: {url}")
    
    # 返回响应的HTML内容
    return resp.text

# 定义了一个函数，用于获取指定URL的内容（以二进制形式）
# 参数：url 表示要获取内容的URL地址
def get_content_of_url(url):
    # 等待1秒，避免请求过于频繁
    time.sleep(1)
    
    # 发送一个GET请求到指定URL，使用之前定义的请求头部
    resp = requests.get(url, headers=headers)
    
    # 如果响应状态码不为200，抛出一个异常
    if resp.status_code != 200:
        raise ValueError(f"Failed to get url: {url}")
    
    # 返回响应的内容（以二进制形式）
    return resp.content

film_info_list = []

for i in range(10):
    html_text = get_html_of_page(i)      # 获取 TOP250 的第 i 页的网页源代码（字符串）
    # 字符串上的方法：index find rfind strip removesuffix   (no select)
    
    soup = bs4.BeautifulSoup(html_text)  # 生成这一页的树状结构（html 根元素）
    
    所有的film_element = soup.select("#content > div > div.article > ol > li")  # 得到这一页上面所有的电影，类型：列表
    
    for 每个film_element in 所有的film_element:
        该电影的编号 = 每个film_element.select("em")[0]
        id = 该电影的编号.get_text()
        print("当前正在处理第", id, "个电影……")
        该电影的封面 = 每个film_element.select("img")[0]
        image_source = 该电影的封面.get("src")

        # with open(f"./covers/{id}.jpg", "wb") as f:
        #     f.write(get_content_of_url(image_source))
        
        所有的title_element = 每个film_element.select("span.title")  # [中文名, 外文名] 或者 [中文名]
        chinese_title = 所有的title_element[0].get_text().strip()   # 取出中文名
        if len(所有的title_element) > 1:                            # 如果有外文名
            foreign_title = 所有的title_element[1].get_text().strip() # 取出外文名
        else:
            foreign_title = "该电影没有外文名"

        # 想把所有的 \x0a 换成空格
        # re.sub("\\x0a", " ", 原始字符串)
        
        该电影的一堆信息 = 每个film_element.select('p[class=""]')[0]  # 演职员表 年份 国家 电影类别
        many_info = 该电影的一堆信息.get_text().strip()
        lines = many_info.split("\n")
        staff = re.sub("\\x0a", " ", lines[0].strip())
        year_nation_category = re.sub("\\x0a", " ", lines[1].strip())
        该电影的评分 = 每个film_element.select("span.rating_num")[0]
        rating = 该电影的评分.get_text()
        该电影的评分人数 = 每个film_element.select("div.star > span:last-child")[0]
        rating_people_count = 该电影的评分人数.get_text()

        if len(每个film_element.select("span.inq")) > 0:
            该电影的一句话评语 = 每个film_element.select("span.inq")[0]
            comment = 该电影的一句话评语.get_text()
        else:
            comment = "该电影没有一句话评语"
        
        # print(id, image_source, staff, year_nation_category, rating, rating_people_count, comment)
        # 将电影信息添加到列表中
        film_info_list.append({
            "id": id,
            "chinese_title": chinese_title,
            "foreign_title": foreign_title,
            "image_source": image_source,
            "staff": staff,
            "year_nation_category": year_nation_category,
            "rating": rating,
            "rating_people_count": rating_people_count,
            "comment": comment
        })

# 储存

with open("./top250-basic.json", "w") as f:
    f.write(json.dumps(film_info_list, ensure_ascii=False, indent=2))


In [5]:
# 加载工具箱
import openpyxl, json

# 从 JSON 文件中读取电影列表
with open("./top250-basic.json", "r") as f:
    film_list = json.loads(f.read())

# 写入 Excel 文件

wb = openpyxl.Workbook()   # 创建了一个新的 Excel 文件
ws = wb.active             # 获取 Excel 文件的第一个 Sheet

# 设置列宽
ws.column_dimensions["B"].width = 20
ws.column_dimensions["C"].width = 30
ws.column_dimensions["D"].width = 35
ws.column_dimensions["E"].width = 20
ws.column_dimensions["G"].width = 12
ws.column_dimensions["H"].width = 20
ws.column_dimensions["I"].width = 35

# 添加表头
ws.append(["id", "中文名", "外文名", "演职员表", "年份 / 国家 / 类别", "评分", "评分人数", "一句话评论", "图片"])

# 遍历电影列表，逐行写入数据
for film in film_list:
    line = [] # Excel 中的新行
    # 向新行里面添加单元格
    line.append(int(film["id"]))
    line.append(film["chinese_title"])
    line.append(film["foreign_title"])
    line.append(film["staff"])
    line.append(film["year_nation_category"])
    line.append(float(film["rating"]))
    line.append(film["rating_people_count"])
    line.append(film["comment"])
    line.append(film["image_source"])
    # 向 sheet 里面添加新行
    ws.append(line)

# 获取当前 sheet 的最大行数和列数
max_row = ws.max_row
max_column = ws.max_column

# 根据条件将 image_source 列的非表头的单元格设置为超链接
for row in range(1, max_row + 1):
    # 获取单元格文本内容
    text = ws.cell(row, 9).value
    # 如果文本内容不是 http 开头就跳过
    if not text.startswith("http"):
        continue
    # 标记为超链接
    make_cell_as_hyperlink(row, 9, text, text)

# 保存 Excel 文件
wb.save("top250.xlsx")

### 赋值计算运算符

在 Python 中，有一些特殊的运算符，如 +=、-=、*=、/=、//= 和 %=，它们可以与赋值操作符结合使用。这些运算符用于对变量进行原地（in-place）操作，即在原始变量的基础上进行特定的运算，并将结果赋值回原始变量。

1. `+=`：加法赋值运算符。它将右侧的值与左侧的变量相加，并将结果赋值给左侧的变量。例如：

```python
a = 5
a += 3 # 相当于 a = a + 3
print(a) # 输出: 8
```

2. `-=`：减法赋值运算符。它将右侧的值从左侧的变量中减去，并将结果赋值给左侧的变量。例如：

```python
a = 5
a -= 3 # 相当于 a = a - 3
print(a) # 输出: 2
```

3. `*=`：乘法赋值运算符。它将右侧的值与左侧的变量相乘，并将结果赋值给左侧的变量。例如：

```python
a = 5
a *= 3 # 相当于 a = a * 3
print(a) # 输出: 15
```

4. `/=`：除法赋值运算符。它将左侧的变量除以右侧的值，并将结果赋值给左侧的变量。例如：

```python
a = 10
a /= 2 # 相当于 a = a / 2
print(a) # 输出: 5.0
```

5. `//=`：整除赋值运算符。它将左侧的变量除以右侧的值，然后向下取整，并将结果赋值给左侧的变量。例如：

```python
a = 10
a //= 3 # 相当于 a = a // 3
print(a) # 输出: 3
```

6. `%=`：取模赋值运算符。它将左侧的变量除以右侧的值，然后取余数，并将结果赋值给左侧的变量。例如：

```python
a = 10
a %= 3 # 相当于 a = a % 3
print(a) # 输出: 1
```

除了上述列举的运算符之外，还有其他一些类似但更不常用的运算符，如逻辑位运算符（&=、|=、^=）、左移位赋值运算符（<<=）、右移位赋值运算符（>>=）等。

### 平均分计算

$$\text{avg}(a_n) = \dfrac{\sum_{i=1}^{n} a_i}{n}=\dfrac{a_1 + a_2 + a_3 + \cdots + a_n}{n}$$

```python
def get_avg(list):
    sum_value = 0
    for number in list:
        sum_value += number   # += 运算符：a += b 等价于 a = a + b
    return sum_value / len(list)
```

### 对列表应用过滤条件生成子列表

```python
def filter_list(lst, filter_func):
    # 过滤列表
    filtered_list = []
    for elem in lst:
        if filter_func(elem):
            filtered_list.append(elem)
    return filtered_list
```

该函数的作用是过滤列表。它接收两个参数：待过滤的列表 list 和过滤条件函数 filter_func。该函数内部遍历列表中的所有元素，并将每个元素传递给过滤函数 filter_func 进行判断。如果 filter_func 返回 True，则判断为应当留存，会将该元素加入到新列表中；否则跳过该元素，不加入到新列表中。最终，返回经过过滤的新列表，完成过滤。

In [7]:
def filter_list(lst, filter_func):
    # 过滤列表
    filtered_list = []
    for elem in lst:
        if filter_func(elem):
            filtered_list.append(elem)
    return filtered_list

# 过滤函数，保留偶数
def is_even(num):
    return num % 2 == 0

# 过滤函数，保留奇数
def is_odd(num):
    return num % 2 == 1

# 测试
my_list = [1, 2, 3, 4, 5, 6]

# 注意是 is_even 不是 is_even()，我们希望将函数本体传递进 filter_list，如果写为 is_even() 那传进去的就是函数的返回值了
# 类比一下政府机关办事员的例子：把 is_even 传进去相当于劳务派遣，整个连屋子带人打包起来交给 filter_list
print(filter_list(my_list, is_even))
print(filter_list(my_list, is_odd))

[2, 4, 6]
[1, 3, 5]


### 统计所有类别

#### 列表去重 - 保证同样的内容在列表里面只出现一次

Python 中的 Set 是一种无序、可变且不重复的集合数据类型。它是由一组不重复的元素组成，类似于数学中的集合概念。

Set 的特点如下：

- Set 中的元素是无序的，不能通过索引访问。
- Set 中的元素是唯一的，不会存在重复的元素。
- Set 是可变的，可以添加和删除元素。

创建 Set 的方式有两种：

- 使用花括号 `{}` 来创建一个**非空**的 Set，例如 `my_set = {1, 2, 3}`。
- 使用 `set()` 函数创建一个空或者非空的 Set，例如 `my_set = set()` 和 `my_set = set([2, 3])`。后者其实是 `set` 函数将一个列表转换成了一个 Set。

#### 使用 Set 对列表去重


In [11]:
my_list = [5, 3, 2, "abc", "abc", 2, 5, 1, 5, 4, 3, "d"]
unique_set = set()  # 创建一个空的集合用于储存元素

for item in my_list:
    unique_set.add(item)  # 将元素逐个添加到集合中。此时，set 中已经存在的元素会被 set 自动忽略

unique_list = list(unique_set)
print(unique_list)   # 已去重，顺序和原数组不一致

[1, 2, 3, 4, 5, 'd', 'abc']


或者

In [13]:
my_list = [5, 3, 2, "abc", "abc", 2, 5, 1, 5, 4, 3, "d"]
unique_set = set(my_list)       # 类型转换：将一个列表转换成了 Set
unique_list = list(unique_set)  # 类型转换：将一个 Set 转换回了列表，此时这个列表已经自动去重完毕
print(unique_list)  # 已去重，顺序和原数组不一致

[1, 2, 3, 4, 5, 'd', 'abc']


### Pyecharts

#### 安装

In [19]:
!pip install pyecharts echarts pyecharts-jupyter-installer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.2 MB/s eta 0:00:00m eta 0:00:010:00:010m
  Preparing metadata (setup.py) ... done
  DEPRECATION: echarts is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for echarts ... done

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


Echarts是一个开源的JavaScript可视化库，用于创建交互式的数据可视化图表。它由百度前端团队开发并维护，支持多种常见的图表类型，如折线图、柱状图、饼图、散点图等。Echarts提供了丰富的配置选项和交互功能，可以根据数据的需求创建各种定制化的图表。

Pyecharts是Echarts的Python封装库，使得使用Python语言可以轻松地生成Echarts图表。Pyecharts提供了一种简单而直观的方式来创建Echarts图表，无需了解复杂的JavaScript代码。它通过Python中的数据结构和方法，将数据转换为Echarts所需的JSON格式，并自动生成对应的HTML文件以展示图表。

Pyecharts具有以下特点：

- 简单易用：使用Python语言进行图表创建，无需编写复杂的JavaScript代码。
- 功能丰富：支持大多数Echarts图表类型和配置选项，满足各种数据可视化需求。
- 高度可定制化：提供丰富的图表配置选项和交互功能，可以对图表进行定制和扩展。

In [3]:
from pyecharts import options as opts
from pyecharts.charts import Bar

# 准备数据
x_data = ["A", "B", "C", "D", "E"]
y_data = [10, 20, 30, 40, 50]

# 创建柱状图实例
bar_chart = Bar()
bar_chart.add_xaxis(x_data)
bar_chart.add_yaxis("柱状图", y_data)

# 设置全局配置项
bar_chart.set_global_opts(title_opts=opts.TitleOpts(title="柱状图示例"))

# 渲染生成HTML文件（可选）
bar_chart.render("bar_chart.html")

'/Users/xiejiss/Code/python/python-preparatory-course/bar_chart.html'